
# NLP Assignment - Question Classification
This notebook outlines the process for data preprocessing, training, and model creation for Question Classification tasks using pretrained Word2Vec embeddings.


In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
import gensim.downloader as api
import random


## Data Preprocessing
Load and preprocess the data for question classification.


In [2]:

# Question Classification Data
train_data = pd.read_csv('../Data/TREC/Raw/train.csv')  # Replace with your path
test_data = pd.read_csv('../Data/TREC/Raw/test.csv')  # Replace with your path
# Preprocess Data (Tokenization, padding, label encoding)
# Add your preprocessing steps here
train_data['text'] = train_data['text'].str.lower()
test_data['text'] = test_data['text'].str.lower()

train_data = train_data.drop(columns=['label-fine'])
test_data = test_data.drop(columns=['label-fine'])

In [3]:
coarse_label = train_data['label-coarse'].unique() # returns an array([])
coarse_label = list(coarse_label) 
selection = random.sample(coarse_label, 2)

In [4]:
for index, row in train_data.iterrows():
    label_coarse = train_data.loc[index, 'label-coarse']
    if label_coarse in selection:
        train_data.loc[index, 'label-coarse'] = 'OTHERS'


for index, row in test_data.iterrows():
    label_coarse = test_data.loc[index, 'label-coarse']
    if label_coarse in selection:
        test_data.loc[index, 'label-coarse'] = 'OTHERS'

C:\Users\leeha\AppData\Local\Temp\ipykernel_10380\603029549.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'OTHERS' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.loc[index, 'label-coarse'] = 'OTHERS'
C:\Users\leeha\AppData\Local\Temp\ipykernel_10380\603029549.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'OTHERS' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_data.loc[index, 'label-coarse'] = 'OTHERS'


In [5]:
train_data

,label-coarse,text
0,OTHERS,how did serfdom develop in and then leave russ...
1,OTHERS,what films featured the character popeye doyle ?
2,OTHERS,how can i find a list of celebrities ' real na...
3,OTHERS,what fowl grabs the spotlight after the chines...
4,2,what is the full form of .com ?
...,...,...
5447,OTHERS,what 's the shape of a camel 's spine ?
5448,OTHERS,what type of currency is used in china ?
5449,4,what is the temperature today ?
5450,4,what is the temperature for cooking ?


In [6]:
train_df, dev_df = train_test_split(train_data, test_size = 500, shuffle = True)

path = '../Data/TREC/Processed/'

# Export DataFrame to a CSV file
train_df.to_csv(f'{path}train.csv', index=False)
dev_df.to_csv(f'{path}dev.csv', index=False)
test_data.to_csv(f'{path}test.csv', index=False)


## Loading Pretrained Word2Vec Embeddings
Load the pretrained Word2Vec model.


In [7]:
path = '../Data/TREC/Processed'

training_dev_df = pd.read_csv(f'{path}/dev.csv')
training_df = pd.read_csv(f'{path}/train.csv')
test_df = pd.read_csv(f'{path}/test.csv')
print(set(training_df['label-coarse']))

{'3', '2', 'OTHERS', '4', '5'}


In [8]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

training_df['label-coarse'] = label_encoder.fit_transform(training_df['label-coarse'])
training_dev_df['label-coarse'] = label_encoder.fit_transform(training_dev_df['label-coarse'])
test_df['label-coarse'] = label_encoder.fit_transform(test_df['label-coarse'])


In [9]:
y_train = to_categorical(training_df['label-coarse'])
y_val = to_categorical(training_dev_df['label-coarse'])
y_test = to_categorical(test_df['label-coarse'])

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(training_df['text'])

X_train_seq = tokenizer.texts_to_sequences(training_df['text'])
X_val_seq = tokenizer.texts_to_sequences(training_dev_df['text'])
X_test_seq = tokenizer.texts_to_sequences(test_df['text'])

MAX_LEN = 50

X_train = pad_sequences(X_train_seq,MAX_LEN)
X_val = pad_sequences(X_val_seq,MAX_LEN)
X_test = pad_sequences(X_test_seq,MAX_LEN)

In [11]:

# Load Pretrained Word2Vec Model
w2v_model = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [12]:
_, embedding_dim = w2v_model.vectors.shape
print (embedding_dim)

300


In [18]:
vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in w2v_model:
        embedding_vec = w2v_model[word]
        if embedding_vec is not None:
            embedding_matrix[i] = embedding_vec
print (embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.08007812  0.10498047  0.04980469 ...  0.00366211  0.04760742
  -0.06884766]
 [ 0.13964844 -0.00616455  0.21484375 ...  0.05712891  0.09960938
  -0.234375  ]
 ...
 [ 0.34570312 -0.05419922 -0.11816406 ... -0.53125     0.12890625
   0.08740234]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.06054688  0.10693359 -0.03662109 ...  0.39257812  0.07666016
  -0.08056641]]



## Model Building


Using Global Average Pooling

In [19]:
from keras.layers import GlobalAveragePooling1D  # Import the necessary layer

model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=embedding_matrix.shape[1],
                    weights=[embedding_matrix],
                    input_length=MAX_LEN,
                    trainable=False))

# Add a Global Average Pooling layer to perform the aggregation
model.add(GlobalAveragePooling1D())

# Add Dense layers (as in your original code)
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [20]:
# Training Question Classification Model
NUM_EPOCH = 200
BATCH_SIZE = 64

model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_data=(X_val, y_val), callbacks= early_stopper, workers= 4)


Epoch 1/200
78/78 [==============================] - 2s 11ms/step - loss: 1.4549 - accuracy: 0.4418 - val_loss: 1.3889 - val_accuracy: 0.3940
Epoch 2/200
78/78 [==============================] - 0s 6ms/step - loss: 1.2741 - accuracy: 0.4479 - val_loss: 1.2713 - val_accuracy: 0.4080
Epoch 3/200
78/78 [==============================] - 1s 6ms/step - loss: 1.1698 - accuracy: 0.5101 - val_loss: 1.1614 - val_accuracy: 0.5180
Epoch 4/200
78/78 [==============================] - 1s 6ms/step - loss: 1.0665 - accuracy: 0.5695 - val_loss: 1.0642 - val_accuracy: 0.5600
Epoch 5/200
78/78 [==============================] - 1s 6ms/step - loss: 0.9740 - accuracy: 0.6323 - val_loss: 0.9753 - val_accuracy: 0.6640
Epoch 6/200
78/78 [==============================] - 0s 6ms/step - loss: 0.8976 - accuracy: 0.6751 - val_loss: 0.9049 - val_accuracy: 0.6840
Epoch 7/200
78/78 [==============================] - 0s 6ms/step - loss: 0.8373 - accuracy: 0.7007 - val_loss: 0.8522 - val_accuracy: 0.7040
Epoch 8/200


In [21]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

16/16 [==============================] - 0s 5ms/step - loss: 0.4138 - accuracy: 0.8680
Test Loss: 0.41378143429756165, Test Accuracy: 0.8679999709129333


Using Global Max Pooling

In [22]:
from keras.layers import GlobalMaxPooling1D  # Import the necessary layer

model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=embedding_matrix.shape[1],
                    weights=[embedding_matrix],
                    input_length=MAX_LEN,
                    trainable=False))

# Add a Global Max Pooling layer to perform the aggregation
model.add(GlobalMaxPooling1D())

# Add Dense layers (as in your original code)
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [23]:
# Training Question Classification Model
NUM_EPOCH = 200
BATCH_SIZE = 64

model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_data=(X_val, y_val), callbacks= early_stopper, workers= 4)

Epoch 1/200
78/78 [==============================] - 2s 12ms/step - loss: 1.3304 - accuracy: 0.4388 - val_loss: 1.2514 - val_accuracy: 0.4460
Epoch 2/200
78/78 [==============================] - 1s 7ms/step - loss: 1.1477 - accuracy: 0.5287 - val_loss: 1.0630 - val_accuracy: 0.5700
Epoch 3/200
78/78 [==============================] - 1s 7ms/step - loss: 1.0122 - accuracy: 0.6036 - val_loss: 0.9386 - val_accuracy: 0.6520
Epoch 4/200
78/78 [==============================] - 1s 6ms/step - loss: 0.9124 - accuracy: 0.6587 - val_loss: 0.8565 - val_accuracy: 0.6880
Epoch 5/200
78/78 [==============================] - 1s 7ms/step - loss: 0.8446 - accuracy: 0.6935 - val_loss: 0.7936 - val_accuracy: 0.7040
Epoch 6/200
78/78 [==============================] - 0s 6ms/step - loss: 0.7957 - accuracy: 0.7102 - val_loss: 0.8032 - val_accuracy: 0.7100
Epoch 7/200
78/78 [==============================] - 1s 6ms/step - loss: 0.7483 - accuracy: 0.7274 - val_loss: 0.7399 - val_accuracy: 0.7060
Epoch 8/200


In [24]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

16/16 [==============================] - 0s 8ms/step - loss: 0.5106 - accuracy: 0.8220
Test Loss: 0.5106096267700195, Test Accuracy: 0.8220000267028809


Using LSTM/RNN 

In [25]:
# Model for Question Classification
# Define question classification model architecture

model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=embedding_matrix.shape[1],
                    weights=[embedding_matrix],
                    input_length=MAX_LEN,  # Length of input sequences
                    trainable=False))  # Set to True if you want to fine-tune embeddings

# Add LSTM layer
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.2))
# Add Dense layer(s)
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=5, activation='softmax'))  # num_classes: number of output classes
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


## Training
Training procedures for both NER and question classification models.


In [26]:
# Training Question Classification Model
NUM_EPOCH = 200
BATCH_SIZE = 64

model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_data=(X_val, y_val), callbacks= early_stopper, workers= 4)


Epoch 1/200
78/78 [==============================] - 15s 137ms/step - loss: 1.0949 - accuracy: 0.5743 - val_loss: 0.7858 - val_accuracy: 0.7440
Epoch 2/200
78/78 [==============================] - 10s 130ms/step - loss: 0.5162 - accuracy: 0.8314 - val_loss: 0.4206 - val_accuracy: 0.8760
Epoch 3/200
78/78 [==============================] - 11s 137ms/step - loss: 0.3582 - accuracy: 0.8716 - val_loss: 0.3777 - val_accuracy: 0.8840
Epoch 4/200
78/78 [==============================] - 10s 130ms/step - loss: 0.2793 - accuracy: 0.9093 - val_loss: 0.3239 - val_accuracy: 0.9000
Epoch 5/200
78/78 [==============================] - 10s 128ms/step - loss: 0.2382 - accuracy: 0.9245 - val_loss: 0.3342 - val_accuracy: 0.8960
Epoch 6/200
78/78 [==============================] - 10s 130ms/step - loss: 0.2069 - accuracy: 0.9281 - val_loss: 0.3322 - val_accuracy: 0.9000
Epoch 7/200
78/78 [==============================] - 10s 135ms/step - loss: 0.1806 - accuracy: 0.9344 - val_loss: 0.2964 - val_accuracy:


## Evaluation
Metrics and evaluation methods for the models.


In [27]:

# Evaluate NER Model
# Add your evaluation code here
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
# Evaluate Question Classification Model
# Add your evaluation code here


16/16 [==============================] - 1s 38ms/step - loss: 0.2971 - accuracy: 0.9260
Test Loss: 0.2971298098564148, Test Accuracy: 0.9259999990463257
